In [ ]:
%load_ext autoreload
%autoreload 2

import cv2
import time
import numpy as np
import sys
from collections import deque
import jsonpickle
from message import Message
from timeit import default_timer as timer

from utils.robot_controller import RobotController

from publisher import Publisher
from utils.keypress_listener import KeypressListener
from rich import print
from utils.utils import load_config

In [ ]:
config = load_config("config.yaml")
robot = RobotController(config)
publisher = Publisher()

In [ ]:
robot.recorder.load_recording("recordings/2023-09-18_17-49-34")

def run():
    if not robot.recorder.playback:
        _, raw_img, cam_fps, img_created = robot.camera.read()
    else:
        cam_fps = 0
        raw_img, speed, turn = next(robot.recorder.get_step)
    
    if raw_img is not None:
        draw_img = raw_img.copy()
        data = robot.run_ekf_slam(raw_img, draw_img)

    return data, cam_fps, draw_img

In [ ]:
count = 0

with robot, publisher:

    while count < 3:
        data, cam_fps, draw_img = run()

        
        time0 = timer()

        msg = Message(
            id = count,
            timestamp = time0,
            start = True,

            landmark_ids = data.landmark_ids,
            landmark_rs = data.landmark_rs,
            landmark_alphas = data.landmark_alphas,
            landmark_positions = data.landmark_positions,

            landmark_estimated_ids = data.landmark_estimated_ids,
            landmark_estimated_positions = data.landmark_estimated_positions,
            landmark_estimated_stdevs = data.landmark_estimated_stdevs,

            robot_position = data.robot_position,
            robot_theta = data.robot_theta,
            robot_stdev = data.robot_stdev,

            text = f"cam fps: {cam_fps}"
        )

        msg_str = jsonpickle.encode(msg)
        publisher.publish_img(msg_str, draw_img)

        count += 1
